In [7]:
import polars as pl

In [8]:
data_poktan = pl.read_csv("data/profil_poktan.csv")
data_poktan.head(5)

PROVINSI,KABUPATEN,KECAMATAN,KELURAHAN,LATITUDE,LONGITUDE,PUS,Kampung KB,Rumah DataKU,BKB,BKR,BKL,UPPKA,PIK-R
str,str,str,str,f64,f64,i64,str,str,str,str,str,str,str
"""SULAWESI BARAT…","""MAMUJU TENGAH""","""TOBADAK""","""MAHAHE""",-2.122958,119.365442,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada"""
"""SULAWESI BARAT…","""MAMUJU TENGAH""","""TOBADAK""","""SALOBAJA""",-2.156691,119.336074,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada"""
"""SULAWESI BARAT…","""MAMUJU TENGAH""","""TOBADAK""","""TOBADAK""",-2.104302,119.491646,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Tidak ada"""
"""SULAWESI BARAT…","""MAMUJU TENGAH""","""TOBADAK""","""PALONGAAN""",-2.105264,119.40054,1,"""Ada""","""Ada""","""Ada""","""Ada""","""Ada""","""Tidak ada""","""Tidak ada"""
"""SULAWESI BARAT…","""MAMUJU TENGAH""","""TOBADAK""","""BATU PARIGI""",-2.07222,119.538128,1,"""Ada""","""Ada""","""Ada""","""Tidak ada""","""Ada""","""Tidak ada""","""Ada"""


In [9]:
# List kolom yang akan dihitung
columns_to_count = ["Kampung KB", "Rumah DataKU", "BKB", "BKR", "BKL", "UPPKA", "PIK-R"]

# Menghitung jumlah "Ada" untuk setiap kolom
counts = [data_poktan[col].str.count_matches("Ada").sum() for col in columns_to_count]

# Menghitung jumlah desa unik berdasarkan kombinasi KECAMATAN dan KELURAHAN
unique_desa = data_poktan.select(["KECAMATAN", "KELURAHAN"]).unique().shape[0]

# Membuat DataFrame hasil
results = pl.DataFrame({
    "Variable": ["Desa/Kelurahan"] + columns_to_count,
    "jumlah_ada": [unique_desa] + counts
})

results

Variable,jumlah_ada
str,i64
"""Desa/Kelurahan…",648
"""Kampung KB""",518
"""Rumah DataKU""",506
"""BKB""",595
"""BKR""",505
"""BKL""",505
"""UPPKA""",430
"""PIK-R""",441


In [10]:
import polars as pl

# Membuat DataFrame dari data yang ada
data = {
    "KABUPATEN": ["MAMUJU TENGAH"] * 5,
    "KECAMATAN": ["TOBADAK"] * 5,
    "KELURAHAN": ["MAHAHE", "SALOBAJA", "TOBADAK", "PALONGAAN", "BATU PARIGI"],
    "LATITUDE": ["-2.122958", "-2.156691", "-2.104302", "-2.105264", "-2.07222"],
    "LONGITUDE": [119.365442, 119.336074, 119.491646, 119.40054, 119.538128],
    "PUS": [1] * 5,
    "Kampung KB": ["Ada"] * 5,
    "Rumah DataKU": ["Tidak ada"] * 5,
    "BKB": ["Ada"] * 5,
    "BKR": ["Ada"] * 5,
    "BKL": ["Tidak ada"] * 5,
    "UPPKA": ["Ada"] * 5,
    "PIK-R": ["Tidak ada"] * 5
}

df = pl.DataFrame(data)

# List kolom yang akan dihitung
columns_to_count = ["Kampung KB", "Rumah DataKU", "BKB", "BKR", "BKL", "UPPKA", "PIK-R"]

# Menghitung jumlah "Ada" untuk setiap kolom dan menyusunnya ke dalam DataFrame baru
results = pl.DataFrame({
    "Variable": columns_to_count,
    "jumlah_ada": [df[col].str.count_matches("Ada").sum() for col in columns_to_count]
})

# Menampilkan hasil
results


Variable,jumlah_ada
str,i64
"""Kampung KB""",5
"""Rumah DataKU""",0
"""BKB""",5
"""BKR""",5
"""BKL""",0
"""UPPKA""",5
"""PIK-R""",0


In [11]:
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

women_bins = np.array([-600, -623, -653, -650, -670, -578, -541, -411, -322, -230])
men_bins = np.array([600, 623, 653, 650, 670, 578, 541, 360, 312, 170])

y = list(range(0, 100, 10))

layout = go.Layout(yaxis=go.layout.YAxis(title='Age'),
                   xaxis=go.layout.XAxis(
                       range=[-1200, 1200],
                       tickvals=[-1000, -700, -300, 0, 300, 700, 1000],
                       ticktext=[1000, 700, 300, 0, 300, 700, 1000],
                       title='Number'),
                   barmode='overlay',
                   bargap=0.1)

data = [go.Bar(y=y,
               x=men_bins,
               orientation='h',
               name='Men',
               hoverinfo='x',
               marker=dict(color='powderblue')
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='Women',
               text=-1 * women_bins.astype('int'),
               hoverinfo='text',
               marker=dict(color='seagreen')
               )]

py.iplot(dict(data=data, layout=layout), filename='EXAMPLES/bar_pyramid')

ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


In [ ]:
import plotly.graph_objs as go
import pandas as pd

# Load the data into a pandas DataFrame
df = pd.read_csv("population_data.csv")

# Create the male and female bar traces
trace_male = go.Bar(x=df[df["gender"]=="M"]["count"],
                	y=df[df["gender"]=="M"]["age"],
                	orientation="h",
                	name="Male",
                	marker=dict(color="#1f77b4"))

trace_female = go.Bar(x=df[df["gender"]=="F"]["count"]*(-1),
                  	y=df[df["gender"]=="F"]["age"],
                  	orientation="h",
                  	name="Female",
                  	marker=dict(color="#d62728"))

# Create the layout
layout = go.Layout(title="Population Pyramid",
               	xaxis=dict(title="Count"),
               	yaxis=dict(title="Age"),
               	barmode="overlay",
               	bargap=0.1)

# Create the figure
fig = go.Figure(data=[trace_male, trace_female], layout=layout)

# Show the plot
fig.show()

In [52]:
kelompok_umur_lk = pl.read_csv('data/PIRAMIDA PENDUDUK - Laki-laki.csv')

# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_lk.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_lk.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_lk = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Laki-laki"
)

# Menambahkan kolom 'Jenis_Kelamin' dengan isi 'Laki-laki'
#melted_result_lk = melted_result.with_columns([pl.lit("Laki-laki").alias("Jenis_Kelamin")])

# Menampilkan hasil
melted_result_lk

PROVINSI,Age_Group,Laki-laki
str,str,i64
"""SULAWESI BARAT…","""Umur 0 - 1""",7655
"""SULAWESI BARAT…","""Umur 2 - 4""",31161
"""SULAWESI BARAT…","""Umur 5 - 9""",59619
"""SULAWESI BARAT…","""Umur 10 - 14""",62885
"""SULAWESI BARAT…","""Umur 15 - 19""",67733
"""SULAWESI BARAT…","""Umur 20 - 24""",63400
"""SULAWESI BARAT…","""Umur 25 - 29""",50836
"""SULAWESI BARAT…","""Umur 30 - 34""",44164
"""SULAWESI BARAT…","""Umur 35 - 39""",43901


In [53]:
kelompok_umur_lk.head(2)

PROVINSI,KABUPATEN,KECAMATAN,KODE,KELURAHAN,Jumlah_Penduduk,Umur 0 - 1,Umur 2 - 4,Umur 5 - 9,Umur 10 - 14,Umur 15 - 19,Umur 20 - 24,Umur 25 - 29,Umur 30 - 34,Umur 35 - 39,Umur 40 - 44,Umur 45 - 49,Umur 50 - 54,Umur 55 - 59,Umur 60 - 64,Umur 65 - 69,Umur 70 - 74,Umur 75+
str,str,str,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""SULAWESI BARAT…","""PASANGKAYU""","""BAMBALAMOTU""",1006,"""BAMBALAMOTU""",1994,52,111,205,217,263,186,157,129,142,135,107,115,76,46,19,20,14
"""SULAWESI BARAT…","""PASANGKAYU""","""BAMBALAMOTU""",2003,"""POLEWALI""",1712,20,96,188,172,213,166,125,107,126,125,107,100,58,53,21,21,14


In [40]:
kelompok_umur_pr = pl.read_csv('data/PIRAMIDA PENDUDUK - Perempuan.csv')
# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_pr.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_pr.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_pr = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Perempuan"
)

# Menambahkan kolom 'Jenis_Kelamin' dengan isi 'Perempuan'
#melted_result_pr = melted_result.with_columns([pl.lit("Perempuan").alias("Jenis_Kelamin")])

# Menampilkan hasil
melted_result_pr

PROVINSI,Age_Group,Perempuan
str,str,i64
"""SULAWESI BARAT…","""Umur 0 - 1""",7111
"""SULAWESI BARAT…","""Umur 2 - 4""",28925
"""SULAWESI BARAT…","""Umur 5 - 9""",55267
"""SULAWESI BARAT…","""Umur 10 - 14""",58918
"""SULAWESI BARAT…","""Umur 15 - 19""",62332
"""SULAWESI BARAT…","""Umur 20 - 24""",60634
"""SULAWESI BARAT…","""Umur 25 - 29""",49743
"""SULAWESI BARAT…","""Umur 30 - 34""",44449
"""SULAWESI BARAT…","""Umur 35 - 39""",44751


In [41]:
df_horizontal_join = melted_result_pr.join(melted_result_lk, on="Age_Group", how="inner")

# Daftar kategori usia
ku = ["0 - 1", "2 - 4", "5 - 9", "10 - 14", "15 - 19", 
      "20 - 24", "25 - 29", "30 - 34", "35 - 39", "40 - 44", 
      "45 - 49", "50 - 54", "55 - 59", "60 - 64", 
      "65 - 69", "70 - 74", "75+"]

# Menghitung berapa kali daftar ku perlu diulang
repeat_count = df_horizontal_join.shape[0] // len(ku) + 1
repeated_ku = (ku * repeat_count)[:df_horizontal_join.shape[0]]

# Menambahkan kolom kategori umur
df_horizontal_join = df_horizontal_join.with_columns([pl.Series(name="Kategori_Umur", values=repeated_ku)])

df_horizontal_join

PROVINSI,Age_Group,Perempuan,PROVINSI_right,Laki-laki,Kategori_Umur
str,str,i64,str,i64,str
"""SULAWESI BARAT…","""Umur 0 - 1""",7111,"""SULAWESI BARAT…",7655,"""0 - 1"""
"""SULAWESI BARAT…","""Umur 2 - 4""",28925,"""SULAWESI BARAT…",31161,"""2 - 4"""
"""SULAWESI BARAT…","""Umur 5 - 9""",55267,"""SULAWESI BARAT…",59619,"""5 - 9"""
"""SULAWESI BARAT…","""Umur 10 - 14""",58918,"""SULAWESI BARAT…",62885,"""10 - 14"""
"""SULAWESI BARAT…","""Umur 15 - 19""",62332,"""SULAWESI BARAT…",67733,"""15 - 19"""
"""SULAWESI BARAT…","""Umur 20 - 24""",60634,"""SULAWESI BARAT…",63400,"""20 - 24"""
"""SULAWESI BARAT…","""Umur 25 - 29""",49743,"""SULAWESI BARAT…",50836,"""25 - 29"""
"""SULAWESI BARAT…","""Umur 30 - 34""",44449,"""SULAWESI BARAT…",44164,"""30 - 34"""
"""SULAWESI BARAT…","""Umur 35 - 39""",44751,"""SULAWESI BARAT…",43901,"""35 - 39"""


In [43]:
y_age = df_horizontal_join['Kategori_Umur'] 
x_M = df_horizontal_join['Laki-laki'] 
x_F = df_horizontal_join['Perempuan'] * -1

if max(x_M) >= max(x_F):
    maks = max(x_M)
else:
    maks = max(x_F)

import math

def round_up_to_nearest(number, base):
    return base * math.ceil(number / base)

def auto_round_up(number):
    if number == 0:
        return 0
    base = 10 ** (len(str(number)) - 1)
    return round_up_to_nearest(number, base)

maks1 = auto_round_up(maks)
maks2 = auto_round_up(int(maks - (maks * 1 / 3)))
maks3 = auto_round_up(int(maks - (maks * 2 / 3)))

tick_vals = [-maks1, -maks2, -maks3, 0, maks1, maks2, maks3]
tick_str = [str(abs(value)) for value in tick_vals]

In [50]:
import plotly.graph_objects as gp
# Creating instance of the figure 
fig = gp.Figure() 
  

# Adding Female data to the figure 
fig.add_trace(gp.Bar(y = y_age, x = x_F, 
                     name = 'Perempuan', orientation = 'h',
                     marker=dict(color='#ffc107'),
                     hovertemplate='Perempuan Umur %{y}<br>Jumlah: %{customdata}<extra></extra>',
                     customdata=[abs(x) for x in x_F]
                     )) 

# Adding Male data to the figure 
fig.add_trace(gp.Bar(y= y_age, x = x_M,  
                     name = 'Laki-laki',  
                     orientation = 'h',
                     marker=dict(color='#0d6efd'),
                     hovertemplate='Laki-laki Umur %{y}<br> %{x}<extra></extra>')) 

  
# Updating the layout for our graph 
fig.update_layout(title = 'Piramida Penduduk', 
                 title_font_size = 22, barmode = 'relative', 
                 bargap = 0.0, bargroupgap = 0, 
                 xaxis = dict(tickvals = tick_vals, 
                              ticktext = tick_str, 
                              title = 'Jumlah', 
                              title_font_size = 14),
                legend=dict(
                        orientation='h',
                        yanchor='bottom',
                        y=-0.3,  # Adjust this value to move the legend up or down
                        xanchor='center',
                        x=0.5
                ),
                plot_bgcolor='#f6f8fa',  # Set background color of the plot area to green
                paper_bgcolor='#f6f8fa'  # Set background color of the entire canvas to green 
)
  
fig.show()

In [51]:
kelompok_umur_lk = pl.read_csv('data/PIRAMIDA PENDUDUK - Laki-laki.csv')
# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_lk.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_lk.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_lk = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Laki-laki"
)

kelompok_umur_pr = pl.read_csv('data/PIRAMIDA PENDUDUK - Perempuan.csv')
# Mendefinisikan range kolom yang akan dijumlahkan
columns_to_sum = [kelompok_umur_pr.columns[i] for i in range(6, 23)]

# Membuat daftar agregasi
aggregations = [pl.sum(col) for col in columns_to_sum]

# Melakukan pengelompokan dan agregasi
result = kelompok_umur_pr.group_by("PROVINSI").agg(aggregations)

# Asumsi bahwa 'result' adalah DataFrame yang dihasilkan dari kode sebelumnya
columns_to_melt = [col for col in result.columns if col != "PROVINSI"]

# Melakukan melt pada DataFrame
melted_result_pr = result.melt(
    id_vars=["PROVINSI"], 
    value_vars=columns_to_melt, 
    variable_name="Age_Group", 
    value_name="Perempuan"
)

df_horizontal_join = melted_result_pr.join(melted_result_lk, on="Age_Group", how="inner")

# Daftar kategori usia
ku = ["0 - 1", "2 - 4", "5 - 9", "10 - 14", "15 - 19", 
      "20 - 24", "25 - 29", "30 - 34", "35 - 39", "40 - 44", 
      "45 - 49", "50 - 54", "55 - 59", "60 - 64", 
      "65 - 69", "70 - 74", "75+"]

# Menghitung berapa kali daftar ku perlu diulang
repeat_count = df_horizontal_join.shape[0] // len(ku) + 1
repeated_ku = (ku * repeat_count)[:df_horizontal_join.shape[0]]

# Menambahkan kolom kategori umur
df_horizontal_join = df_horizontal_join.with_columns([pl.Series(name="Kategori_Umur", values=repeated_ku)])

y_age = df_horizontal_join['Kategori_Umur'] 
x_M = df_horizontal_join['Laki-laki'] 
x_F = df_horizontal_join['Perempuan'] * -1

if max(x_M) >= max(x_F):
    maks = max(x_M)
else:
    maks = max(x_F)

import math

def round_up_to_nearest(number, base):
    return base * math.ceil(number / base)

def auto_round_up(number):
    if number == 0:
        return 0
    base = 10 ** (len(str(number)) - 1)
    return round_up_to_nearest(number, base)

maks1 = auto_round_up(maks)
maks2 = auto_round_up(int(maks - (maks * 1 / 3)))
maks3 = auto_round_up(int(maks - (maks * 2 / 3)))

tick_vals = [-maks1, -maks2, -maks3, 0, maks1, maks2, maks3]
tick_str = [str(abs(value)) for value in tick_vals]

import plotly.graph_objects as gp
# Creating instance of the figure 
fig = gp.Figure() 
  

# Adding Female data to the figure 
fig.add_trace(gp.Bar(y = y_age, x = x_F, 
                     name = 'Perempuan', orientation = 'h',
                     marker=dict(color='#ffc107'),
                     hovertemplate='Perempuan Umur %{y}<br>Jumlah: %{customdata}<extra></extra>',
                     customdata=[abs(x) for x in x_F]
                     )) 

# Adding Male data to the figure 
fig.add_trace(gp.Bar(y= y_age, x = x_M,  
                     name = 'Laki-laki',  
                     orientation = 'h',
                     marker=dict(color='#0d6efd'),
                     hovertemplate='Laki-laki Umur %{y}<br> %{x}<extra></extra>')) 

  
# Updating the layout for our graph 
fig.update_layout(title = 'Piramida Penduduk', 
                 title_font_size = 22, barmode = 'relative', 
                 bargap = 0.0, bargroupgap = 0, 
                 xaxis = dict(tickvals = tick_vals, 
                              ticktext = tick_str, 
                              title = 'Jumlah', 
                              title_font_size = 14),
                legend=dict(
                        orientation='h',
                        yanchor='bottom',
                        y=-0.3,  # Adjust this value to move the legend up or down
                        xanchor='center',
                        x=0.5
                ),
                plot_bgcolor='#f6f8fa',  # Set background color of the plot area to green
                paper_bgcolor='#f6f8fa'  # Set background color of the entire canvas to green 
)
  
fig.show()

In [56]:
df_horizontal_join.select('Kategori_Umur', 'Perempuan', 'Laki-laki')

Kategori_Umur,Perempuan,Laki-laki
str,i64,i64
"""0 - 1""",7111,7655
"""2 - 4""",28925,31161
"""5 - 9""",55267,59619
"""10 - 14""",58918,62885
"""15 - 19""",62332,67733
"""20 - 24""",60634,63400
"""25 - 29""",49743,50836
"""30 - 34""",44449,44164
"""35 - 39""",44751,43901
